In [1]:

from src.analysis.aggregate_trial_results import aggregate_trial_results
from src.analysis.repeated_measures_anova import (
    run_repeated_measures_anova,
    print_rm_anova_summary,
    prepare_repeated_measures_anova_exp2
)

# Create summary stats using default results folder path
aggregate_trial_results = aggregate_trial_results()

# Prepare data for analysis
repeated_df_31 = prepare_repeated_measures_anova_exp2(aggregate_trial_results, 'llama_31', verbose=True)

# Run the repeated measures ANOVA
anova_results_31 = run_repeated_measures_anova(repeated_df_31)

Aggregated trial results saved to /workspaces/coordination/pipeline/4_analysis/trial_results_aggregated.csv

=== Rows with unanswered_prop above threshold ===
        model_name task_instruction   task_options  unanswered_prop
188    llama-31-8b       coordinate  shapes-3-text         0.208333
45   llama-31-405b          control        letters         0.350000

=== Invalid metric rows for top_prop_all ===
        model_name task_instruction  task_options  top_prop_all
321    llama-31-8b          control       colours           0.0
251    llama-31-8b          control  colours-text           0.0
162   llama-31-70b          control       colours           0.0
14    llama-31-70b          control  colours-text           0.0
98   llama-31-405b          control  colours-text           0.0
315  llama-31-405b          control       colours           0.0

=== Invalid metric rows for convergence_answered ===
        model_name task_instruction  task_options  convergence_answered
321    llama-31-8

ValueError: Data validation failed. Use verbose=True for details.

In [2]:
#overview
print_rm_anova_summary(anova_results_31)


=== Repeated Measures ANOVA Summary ===

TOP_PROP:

Descriptive Statistics:
+---------------------+---------+--------+-------+
|                     |   count |   mean |   std |
+=====================+=========+========+=======+
| ('405B', 'with')    |      20 |  0.623 | 0.189 |
+---------------------+---------+--------+-------+
| ('405B', 'without') |      20 |  0.582 | 0.165 |
+---------------------+---------+--------+-------+
| ('70B', 'with')     |      20 |  0.528 | 0.134 |
+---------------------+---------+--------+-------+
| ('70B', 'without')  |      20 |  0.529 | 0.131 |
+---------------------+---------+--------+-------+
| ('8B', 'with')      |      20 |  0.36  | 0.052 |
+---------------------+---------+--------+-------+
| ('8B', 'without')   |      20 |  0.438 | 0.122 |
+---------------------+---------+--------+-------+

Model Size Effect:
F(2,38) = 16.52, p = 6.88e-06, η² = 0.278
→ strong evidence was found for a large effect of model size

Reasoning Effect:
F(1,19) = 0.68, 

In [3]:
from src.analysis.visualization import (
    plot_model_comparison,
    create_comprehensive_analysis,
)
from src.analysis.df_formatting import print_nice_dataframe
# create_comprehensive_analysis(repeated_df_31)
# plot_model_comparison(repeated_df_31)
print_nice_dataframe(repeated_df_31)

Displaying first 20 rows (total: 60):

+---------------+----------------------+---------------------------------+------------------------------+------------------------------+---------------------------+--------------+
| model         | task_options         |   convergence_without_reasoning |   convergence_with_reasoning |   top_prop_without_reasoning |   top_prop_with_reasoning | model_size   |
+===============+======================+=================================+==============================+==============================+===========================+==============+
| llama-31-405b | colours              |                        0.275694 |                     0.257222 |                     0.391667 |                  0.391667 | 405B         |
+---------------+----------------------+---------------------------------+------------------------------+------------------------------+---------------------------+--------------+
| llama-31-405b | colours-text         |                     